In [89]:
from dataclasses import dataclass
import dask.array as da
from prefect import task, flow
from pycromanager import Dataset
import numpy as np
from prefect.tasks import task_input_hash
from datetime import timedelta




In [ ]:
@dataclass
class Data:
    local_dataset_path: str = None
    nas_dataset_path: str = None
    images: da = None
    masks: dict = None
    props: dict = None

    def __repr__(self):
        return (f"Data(local_dataset_path={self.local_dataset_path}, "
                f"nas_dataset_path={self.nas_dataset_path}, "
                f"images_shape={self.images.shape if self.images is not None else 'None'}, "
                f"masks_keys={list(self.masks.keys()) if self.masks is not None else 'None'}, "
                f"props_keys={list(self.props.keys()) if self.props is not None else 'None'})")

    def cache_key(self):
        # Generate a unique cache key based on relevant attributes
        return (self.local_dataset_path, self.images.shape if self.images is not None else None)

    def __getitem__(self, key):
        """
        This allows access to the data at a specific position (p) and timepoint (t).
        The `key` is expected to be a tuple (p, t).
        """
        p, t = key  # Unpack the position and timepoint
        if self.images is None:
            raise ValueError("Images not loaded. Ensure images are loaded before accessing.")
        
        # Return the image slice at position p and timepoint t
        return self.images[p, t]

    def __setitem__(self, key, value):
        """
        This allows setting a value for a specific position (p) and timepoint (t).
        The `key` is expected to be a tuple (p, t).
        """
        p, t = key  # Unpack the position and timepoint
        if self.images is None:
            raise ValueError("Images not loaded. Ensure images are loaded before setting.")
        
        # Set the image slice at position p and timepoint t
        self.images[p, t] = value



In [ ]:
# Set cache to use pickle instead of json
@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), result_serializer='pickle')
def download_data(nas_dataset_location: str) -> str:
    # Simulate downloading the dataset
    return f"Downloaded data from {nas_dataset_location}"

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), result_serializer='pickle')
def load_in_data(data: Data) -> Data:
    # Load the dataset using pycromanager
    ds = Dataset(data.local_dataset_path)
    images = ds.as_array(['position', 'time', 'channel', 'z'])

    # Initialize the Data object

    return images, ds

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), result_serializer='pickle')
def generate_masks(data: Data) -> Data:
    masks = {}

    # Loop through position and time dimensions (p, t)
    for p in range(data.images.shape[0]):  # Loop over positions (p)
        for t in range(data.images.shape[1]):  # Loop over timepoints (t)
            # Access the 2D image at position p, timepoint t using the Data object
            image_at_p_t = data[(p, t)]  # Use __getitem__

            # Generate mask for this image (This is a placeholder for your actual mask generation logic)
            mask = da.ones(image_at_p_t.shape) # Example mask

            # Store the mask with a key based on the position and timepoint
            masks[(p, t)] = mask

    data.masks = masks
    return data

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), result_serializer='pickle')
def measure_props(data: Data) -> Data:
    data.props = {}

    # Loop through position and time dimensions (p, t)
    for p in range(data.images.shape[0]):  # Loop over positions (p)
        for t in range(data.images.shape[1]):  # Loop over timepoints (t)
            # Access the 2D image at position p, timepoint t using the Data object
            image_at_p_t = data[(p, t)]  # Use __getitem__

            data.props[f'mean_intensity_{p}_{t}'] = float(np.mean(image_at_p_t)) # Example property measurement
    return data

@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1), result_serializer='pickle')
def save_results(data: Data):
    # Simulate saving the results (you can customize this to actually save results)
    print(f"Saving results for dataset {data.local_dataset_path}")
    print(f"Properties: {data.props}")
    print(f"Masks: {data.masks}")
    return None

In [99]:
@flow
def image_processor(data: Data):
    # Load the data
    data = load_in_data(data)

    # Generate masks for the images
    data = generate_masks(data)

    # Measure properties from the images and masks
    data = measure_props(data)

    # Save the results
    save_results(data)

    return data





c:\Users\formanj\GitHub\AngelFISH\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [100]:
data = Data(local_dataset_path=r'C:\Users\formanj\GitHub\AngelFISH\dataBases\rep1_1hr_tilingManyFOVs_2')
image_processor(data)

c:\Users\formanj\GitHub\AngelFISH\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


12:00:51.802 | INFO    | Flow run 'vegan-numbat' - Beginning flow run 'vegan-numbat' for flow 'image-processor'

Dataset opened                


12:00:52.276 | WARNING | Task run 'load_in_data-8d2' - Encountered an error while serializing result for transaction 'ecef760c733581e12f8beed3bdf38124': Failed to serialize object of type 'Data' with serializer 'pickle'. You can try a different serializer (e.g. result_serializer="json") or disabling persistence (persist_result=False) for this flow or task. Code execution will continue, but the transaction will not be committed.

12:00:52.279 | INFO    | Task run 'load_in_data-8d2' - Finished in state Completed()

...

12:00:53.940 | INFO    | Task run 'measure_props-37d' - Finished in state Completed()

Saving results for dataset C:\Users\formanj\GitHub\AngelFISH\dataBases\rep1_1hr_tilingManyFOVs_2
Properties: {'mean_intensity_0_0': 1288.9914207458496, 'mean_intensity_1_0': 1895.1547660827637, 'mean_intensity_2_0': 1350.2858963012695, 'mean_intensity_3_0': 1694.776840209961, 'mean_intensity_4_0': 1199.299373626709, 'mean_intensity_5_0': 2717.7862281799316, 'mean_intensity_6_0': 2108.0496711730957, 'mean_intensity_7_0': 2752.9861450195312, 'mean_intensity_8_0': 2503.7841262817383, 'mean_intensity_9_0': 3111.8259315490723, 'mean_intensity_10_0': 3568.5727195739746, 'mean_intensity_11_0': 3670.27921295166, 'mean_intensity_12_0': 1657.8752822875977, 'mean_intensity_13_0': 2463.369342803955, 'mean_intensity_14_0': 3294.025924682617, 'mean_intensity_15_0': 3093.895824432373, 'mean_intensity_16_0': 631.9370536804199, 'mean_intensity_17_0': 1453.6850242614746, 'mean_intensity_18_0': 2383.6271362304688, 'mean_intensity_19_0': 1385.6825103759766, 'mean_intensity_20_0': 1447.9665756225586, 'mean

12:00:54.408 | INFO    | Task run 'save_results-20b' - Finished in state Completed()

12:00:54.459 | INFO    | Flow run 'vegan-numbat' - Finished in state Completed()

Data(local_dataset_path=C:\Users\formanj\GitHub\AngelFISH\dataBases\rep1_1hr_tilingManyFOVs_2, nas_dataset_path=None, images_shape=(99, 1, 1, 1, 512, 512), masks_keys=[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (51, 0), (52, 0), (53, 0), (54, 0), (55, 0), (56, 0), (57, 0), (58, 0), (59, 0), (60, 0), (61, 0), (62, 0), (63, 0), (64, 0), (65, 0), (66, 0), (67, 0), (68, 0), (69, 0), (70, 0), (71, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (78, 0), (79, 0), (80, 0), (81, 0), (82, 0), (83, 0), (84, 0), (85, 0), (86, 0), (87, 0), (88, 0), (89, 0), (90, 0), (91, 0), (92, 0), (93, 

In [94]:
print(data.masks)

{(0, 0): array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]]]), (1, 0): array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]]]), (2, 0): array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]]]), (3, 0): array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1